In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob

import pickle
from pydantic import BaseModel
from openai import OpenAI

from tenacity import (
                        retry,
                        stop_after_attempt,
                        wait_random_exponential
)

from tqdm import tqdm

In [10]:
def generate_predictions(system_msg, user_msg, client):
    
    result = execute_prompt(client, system_msg, user_msg)
    jd_prediction = result.choices[0].message.content.split('Answer: ')[-1].strip()
    
    return jd_prediction

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
path = '/home/manoelflorencio/cta_for_jd/testbedXS'
os.chdir(path)
print(os.getcwd())

/home/manoelflorencio/cta_for_jd/testbedXS


In [4]:
@retry(wait=wait_random_exponential(min=1,max=60), stop=stop_after_attempt(6))
def execute_prompt(client, system_msg, user_msg):
    completion = client.chat.completions.create(
                                            model="gpt-4o",
                                            messages=[
                                                        {
                                                            "role": "system", 
                                                             "content": f"{system_msg}"
                                                        },
                                                        {
                                                            "role": "user",
                                                            "content": f"{user_msg}"
                                                        }
                                                    ]
                                            )
    return completion

In [5]:
client = OpenAI()

In [7]:
prompts = np.load('Description_test/table_descriptions_test_2025_01_05/Column_JOIN_Prompts/column_JD.npy')
prompts.shape

(119, 2)

In [11]:
prompts[0][1]

'Target description:     The full postal address of the community garden or food tree, combining street number, name, direction, and type for complete location details.\n                   Candidate description:  The ADDRESS column provides the street address of each community centre. These addresses include street numbers and names, such as 1 Kingsway for Mount Pleasant and 1318 Cartwright St for False Creek.\n                   Answer:'

In [12]:
asnwers = []

for i in tqdm(range(len(prompts))):

    system_msg, user_msg = prompts[i][0], prompts[i][1]
    result = generate_predictions(system_msg, user_msg, client)
    asnwers.append(result)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [01:01<00:00,  1.94it/s]


In [13]:
asnwers

['Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes']

In [15]:
results_df = pd.read_csv('Description_test/table_descriptions_test_2025_01_05/junio_JD_matches_joinable_tables.csv')
results_df = results_df[results_df['JOINABLE_TD_v2'] == 'Yes']
results_df.shape

(119, 9)

In [16]:
results_df['COLUMN_JOINABLE'] = asnwers

In [17]:
results_df

,LEFT_TABLE,LEFT_COLUMN,LEFT_DESCRIPTION,RIGHT_TABLE,RIGHT_COLUMN,RIGHT_DESCRIPTION,JOINABLE,JOINABLE_TD,JOINABLE_TD_v2,COLUMN_JOINABLE
0,community-centres,ADDRESS,The full postal address of the community garde...,community-gardens-and-food-trees,MERGED_ADDRESS,The ADDRESS column provides the street address...,Yes,Yes,Yes,Yes
1,community-centres,Geo Local Area,The neighbourhood or broader geographical area...,community-gardens-and-food-trees,Geo Local Area,The Geo Local Area column identifies the gener...,Yes,Yes,Yes,Yes
2,community-centres,ADDRESS,This column contains the street address of the...,cultural-spaces,ADDRESS,The ADDRESS column provides the street address...,Yes,Yes,Yes,No
3,community-centres,Geo Local Area,This column identifies the local area or neigh...,cultural-spaces,LOCAL_AREA,The Geo Local Area column identifies the gener...,Yes,Yes,Yes,Yes
6,community-centres,Geo Local Area,"In this column, the local areas or neighborhoo...",libraries,Geo Local Area,The Geo Local Area column identifies the gener...,Yes,Yes,Yes,Yes
7,community-centres,Geo Local Area,The neighborhood within Vancouver where the ar...,public-art,Neighbourhood,The Geo Local Area column identifies the gener...,Yes,No,Yes,Yes
8,community-centres,Geo Local Area,A broader geographical classification indicati...,public-art,GeoLocalArea,The Geo Local Area column identifies the gener...,Yes,No,Yes,Yes
9,community-centres,Geo Local Area,The broader neighborhood or community area whe...,rental-standards-current-issues,Geo Local Area,The Geo Local Area column identifies the gener...,Yes,Yes,Yes,Yes
10,community-centres,Geo Local Area,The Geo Local Area column indicates the neighb...,schools,Geo Local Area,The Geo Local Area column identifies the gener...,Yes,Yes,Yes,Yes
11,community-centres,Geo Local Area,This column specifies the local area or neighb...,street-intersections,Geo Local Area,The Geo Local Area column identifies the gener...,Yes,Yes,Yes,Yes


In [18]:
results_df.to_csv('Description_test/table_descriptions_test_2025_01_05/column_jd.csv')